In [1]:
import os
import pandas as pd
from itertools import islice
from amplpy import AMPL, Environment
from IPython.display import display

**File Paths**

In [2]:
baseexppath = '/Users/Danny/Desktop/CATEGORIES/CAREER_MANAGEMENT/CRC_ResearchScientist_Optimization/Optimization_Tool/2_ExperimentFolder/'
amplappdir = os.path.join(baseexppath, 'ampl/amplide.macosx64/')
projectpath = os.path.join(baseexppath, 'ampl/OptEfficiencySubProblem/')
datapath = os.path.join(baseexppath, 'ampl/OptEfficiencySubProblem/data/')

# Specify model and data files
# f_mod = os.path.join(baseexppath, 'ampl/example/steel3.mod')
f_mod = os.path.join(projectpath, 'test4.mod')
# f_dat = os.path.join(baseexppath, 'ampl/example/steel3.dat')
f_dat = os.path.join(projectpath, 'test4.dat')

# Specify AMPL solver to be used
f_minos_solver = os.path.join(projectpath, 'amplide.macosx64/minos')
f_gurobi_solver = os.path.join(projectpath, 'amplide.macosx64/gurobi')
f_solver = f_minos_solver

# Data table directories
sourcedatadir = os.path.join(baseexppath, 'OptSandbox/data/test_source/')
metadatadir = os.path.join(baseexppath, 'OptSandbox/data/test_metadata/')

**Set up the AMPL environment**

In [3]:
ampl = AMPL(Environment(amplappdir))
ampl.setOption('solver', f_solver)
value = ampl.getOption('solver')
print(value)

/Users/Danny/Desktop/CATEGORIES/CAREER_MANAGEMENT/CRC_ResearchScientist_Optimization/Optimization_Tool/2_ExperimentFolder/ampl/OptEfficiencySubProblem/amplide.macosx64/minos


**Read in the model**

In [4]:
ampl.read(f_mod)

**Read in the data tables**

In [5]:
# Data tables for the set definitions
TblBmp = pd.read_csv(os.path.join(sourcedatadir, 'TblBmp.csv'))
TblBmpGroup = pd.read_csv(os.path.join(sourcedatadir, 'TblBmpGroup.csv'))
TblBmpLoadSourceGroup = pd.read_csv(os.path.join(sourcedatadir, 'TblBmpLoadSourceGroup.csv'))
TblBmpType = pd.read_csv(os.path.join(sourcedatadir, 'TblBmpType.csv'))

TblLoadSource = pd.read_csv(os.path.join(sourcedatadir, 'TblLoadSource.csv'), skipinitialspace=True)
TblLoadSource['loadsource'] = TblLoadSource['loadsource'].str.strip() # There is an extra space after "Specialty Crop Low" that needs to be removed.

TblLoadSourceGroup = pd.read_csv(os.path.join(sourcedatadir, 'TblLoadSourceGroup.csv'))
TblLoadSourceGroupLoadSource = pd.read_csv(os.path.join(sourcedatadir, 'TblLoadSourceGroupLoadSource.csv'))
TblLandRiverSegment = pd.read_csv(os.path.join(sourcedatadir, 'TblLandRiverSegment.csv'))

TblGeography = pd.read_csv(os.path.join(sourcedatadir, 'TblGeography.csv'))
TblGeographyLrSeg = pd.read_csv(os.path.join(sourcedatadir, 'TblGeographyLrSeg.csv'))
TblGeographyType = pd.read_csv(os.path.join(sourcedatadir, 'TblGeographyType.csv'))

# Data tables for the parameter definitions
TblCostBmpLand = pd.read_csv(os.path.join(metadatadir, 'TblCostBmpLand.csv'))
TblBmpEfficiency = pd.read_csv(os.path.join(sourcedatadir, 'TblBmpEfficiency.csv'))
# Target load reductions ???  (set this ourselves??)
TblLandUsePreBmp = pd.read_csv(os.path.join(sourcedatadir, 'TblLandUsePreBmp.csv'))
Tbl2010NoActionLoads = pd.read_csv(os.path.join(datapath, '2010NoActionLoads.csv'))

# Data table generated by separate python script, the set of load source *groups* where each load source *group* contains one and only one load source
singlelsgrpdf = pd.read_csv(os.path.join(datapath, 'single-ls_groups.csv'))

**Instance specifiers**

$y$, the scenario base year (defines the available load source acres ($T$) and their base loads ($\phi$))<br>
$\kappa$, the cost profile (defines the costs ($c$))

In [6]:
baseconditionid = 29
costprofileid=4

**SETS**

**Pollutants and Land river segments**

$P$, the set of pollutants $p=\{nitrogen, phosphorous, sediment\}$ <br>
$L$, a set of land river segments $l$

In [7]:
pltnts = ampl.getSet('PLTNTS')
pltnts.setValues(['N', 'P', 'S'])
print(pltnts.getValues())

lrsegs = ampl.getSet('LRSEGS')
lrsegids = TblLandRiverSegment[TblLandRiverSegment['landriversegment']=='N51133RL0_6450_0000'].lrsegid.tolist()
lrsegs.setValues(lrsegids)
print(lrsegs.getValues())
lrsegsetlist = list([int(x) for x in lrsegs.getValues().toPandas().index])

   PLTNTS   
    'N'        
    'P'        
    'S'        

   LRSEGS   
    1677       



**Load Sources and their groups**

$\Lambda$, the set of load sources $\lambda$ <br>
$\Psi$, the set of all load source *groups* $\psi$, where $\psi=\{\lambda_{1}, \lambda_{2}...\lambda_{m_{\psi}}\}\subseteq\Lambda$ <br>
$\Psi^{*}$, the set of load source *groups* where each load source *group* contains one and only one load source ($\Psi^{*}\subset\Psi\quad\mid\quad\left\vert\psi^{*}\right\vert=1 \quad\forall \psi^{*}\in\Psi^{*} $) <br>
$\psi_{\lambda}^{*}$, the load source *group* containing only the load source $\lambda$ <br>

*Furthermore*, limit the load sources set to only include those that have base loading rates defined in the No Action data, and only those that have an acreage defined in TblLandUsePreBmp

In [8]:
loadsrcs = ampl.getSet('LOADSRCS')
# limit the load sources set to only include those that have 
#  base loading rates defined in the No Action data, 
#  and only those that have an acreage defined in TblLandUsePreBmp
df = TblLandUsePreBmp[(TblLandUsePreBmp['baseconditionid']==baseconditionid) &\
                      (TblLandUsePreBmp['lrsegid'].isin(lrsegsetlist))].copy()
df = singlelsgrpdf[singlelsgrpdf['loadsourceid'].isin(df['loadsourceid'])]
loadsrcs.setValues(df.loadsourceid.tolist())
loadsrcsetlist = list([int(x) for x in loadsrcs.getValues().toPandas().index])
#print(loadsrcs.getValues())

loadsrcgrps = ampl.getSet('LOADSRCGRPS')
loadsrcgrps.setValues(TblLoadSourceGroup.loadsourcegroupid.tolist())
#print(loadsrcgrps.getValues())

loadsrcgrping = ampl.getSet('LOADSRCGRPING')
loadsrcgrping.setValues(list(zip(TblLoadSourceGroupLoadSource.loadsourcegroupid.tolist(),
                             TblLoadSourceGroupLoadSource.loadsourceid.tolist())))
#print(loadsrcgrping.getValues())

**BMPs and their groups**

$B$, a set of BMPs $b$ <br>
$\Gamma$, a set of BMP *groups* $\gamma$, where $\gamma=\{b_{1}, b_{2}...b_{n_{\gamma}}\}\subseteq B$<br>
$\gamma_{b}$, the BMP *group* to which BMP $b$ belongs

In [9]:
bmps = ampl.getSet('BMPS')
bmps.setValues(TblBmp.bmpid.tolist())
# print(bmps.getValues())

bmpgrps = ampl.getSet('BMPGRPS')
bmpgrpdf = TblBmpGroup[TblBmpGroup['ruleset']=='spbmpruleset_efficiencybmps']
bmpgrps.setValues(bmpgrpdf.bmpgroupid.tolist())
# print(bmpgrps.getValues())

bmpgrping = ampl.getSet('BMPGRPING')
bmpgrping.setValues(list(zip(TblBmp.bmpid.tolist(),
                             bmpgrpdf.bmpgroupid.tolist())))
# print(bmpgrping.getValues())

**PARAMETERS**

$c_{b}^{\kappa}$, the cost per acre of BMP $b$ (for cost profile $\kappa$) <br>

In [10]:
c = ampl.getParameter('c')
df = TblCostBmpLand[TblCostBmpLand['costprofileid']==costprofileid]
c.setValues(dict(zip(df.bmpid, df.totalannualizedcostperunit)))
# print(c.getValues())

$E_{b,p,l,\lambda}$, the effectiveness per acre of BMP $b$ on reducing pollutant $p$, in land-river segment $l$ and load source $\lambda$ <br>

In [11]:
# Some pre-processing is necessary to build the parameter dictionary
effsubtable = TblBmpEfficiency[TblBmpEfficiency['lrsegid'].isin(lrsegsetlist)]
# make the pollutant names into an index instead of separate columns
listofdataframes = []
pltntdict = {'tn': 'N', 'tp': 'P', 'sed': 'S'}
for ps in ['tn', 'tp', 'sed']:
    bmpeff = effsubtable.loc[:, ['bmpid', 'lrsegid', 'loadsourceid', ps]]
    bmpeff['pltnt'] = pltntdict[ps]
    bmpeff.rename(columns={ps: 'effvalue'}, inplace=True)
    listofdataframes.append(bmpeff)
df = pd.concat(listofdataframes)

# Retain only those effectivenesses pertaining to loadsources in our set
df = df[df['loadsourceid'].isin(loadsrcsetlist)]

# Convert groups to dictionary ( with tuple->value structure ) 
grouped = df.groupby(['bmpid', 'pltnt', 'lrsegid', 'loadsourceid'])
Edict = grouped['effvalue'].apply(lambda x: list(x)[0]).to_dict()

# display 5 keys for illustration
nrandkeys = list(islice(Edict,5))
for k, v in zip(nrandkeys, [Edict[x] for x in nrandkeys]):
    print(k, v)
    
E = ampl.getParameter('E')
E.setValues(Edict)

(3, 'N', 1677, 1) 0.03
(3, 'N', 1677, 2) 0.08
(3, 'N', 1677, 3) 0.08
(3, 'N', 1677, 4) 0.08
(3, 'N', 1677, 5) 0.03


$\tau_{l,p}$, the target percent load reduction per pollutant per land river segment

In [12]:
tau = ampl.getParameter('tau')

Taudict = {}
for l in lrsegsetlist:
    Taudict[(l, 'N')] = 0.5
    Taudict[(l, 'P')] = 0.5
    Taudict[(l, 'S')] = 0.5
display(Taudict)

tau.setValues(Taudict)

{(1677, 'N'): 0.5, (1677, 'P'): 0.5, (1677, 'S'): 0.5}

$\phi_{l,\lambda,p}^{y}$, the base nutrient load per pollutant per load source per land river segment (for year $y$)

In [13]:
# Some pre-processing is necessary to build the parameter dictionary

# Unfortunately, the NoActionLoads table is from the website so doesn't have id numbers.  Let's translate this table to id numbers...

# Let's make sure the columns are all lowercase
Tbl2010NoActionLoads.columns = map(str.lower, Tbl2010NoActionLoads.columns)

# First, let's translate our lrseg list to the fullnames so we can subset it before translating.
gtypeid = TblGeographyType[TblGeographyType['geographytypefullname']=='Land River Segment indicating if in or out of CBWS'].geographytypeid.tolist()[0]
geolrsegsubtbl = TblGeographyLrSeg.loc[TblGeographyLrSeg['lrsegid'].isin(lrsegsetlist)]
geosubtbl = geolrsegsubtbl.merge(TblGeography, on='geographyid', how='inner')
lrsegfullnames = geosubtbl[geosubtbl['geographytypeid']==gtypeid].geographyfullname.tolist()

loadssubtbl = Tbl2010NoActionLoads[Tbl2010NoActionLoads['geography'].isin(lrsegfullnames)]

# Go from load source table with geographyfullname to geographyid
includecols = ['geography', 'loadsource', '2010 no action_nloadeos', '2010 no action_ploadeos', '2010 no action_sloadeos']
loadssubtbl = loadssubtbl.loc[:, includecols].merge(TblGeography, how='inner',
                                                    left_on='geography',
                                                    right_on='geographyfullname')
loadssubtbl.drop(columns=['geographyname',
                          'geographyfullname',
                          'geography',
                          'geographytypeid'], inplace=True)

# Go from load source table with geographyid to lrsegid
includecols = ['geographyid', 'lrsegid']
loadssubtbl = TblGeographyLrSeg.loc[:,includecols].merge(loadssubtbl, how='inner',
                                                         on='geographyid')
loadssubtbl.drop(columns=['geographyid'], inplace=True)

# Go from LoadSource to loadsourceid
includecols = ['loadsourceid', 'loadsource']
loadssubtbl = TblLoadSource.loc[:,includecols].merge(loadssubtbl, how='inner',
                                                     on='loadsource')
loadssubtbl.drop(columns=['loadsource'], inplace=True)
display(loadssubtbl.head(2))

# only retain loadsources that are represented by a single-ls loadsource group.
loadssubtbl = loadssubtbl[loadssubtbl['loadsourceid'].isin(loadsrcsetlist)]

# make the pollutant names into an index instead of separate columns
listofdataframes = []
pcolnames = ['2010 no action_nloadeos', '2010 no action_ploadeos', '2010 no action_sloadeos']
pltntdict = {pcolnames[0]: 'N',
             pcolnames[1]: 'P',
             pcolnames[2]: 'S'}
for ps in [pcolnames[0], pcolnames[1], pcolnames[2]]:
    llload = loadssubtbl.loc[:, ['lrsegid', 'loadsourceid', ps]]
    llload['pltnt'] = pltntdict[ps]
    llload.rename(columns={ps: 'loadratelbsperyear'}, inplace=True)
    listofdataframes.append(llload)
df = pd.concat(listofdataframes)
# Convert groups to dictionary ( with tuple->value structure ) 
grouped = df.groupby(['lrsegid', 'loadsourceid', 'pltnt'])
LoadDict = grouped['loadratelbsperyear'].apply(lambda x: list(x)[0]).to_dict()

# display 5 keys for illustration
nrandkeys = list(islice(LoadDict,5))
for k, v in zip(nrandkeys, [LoadDict[x] for x in nrandkeys]):
    print(k, v)
    
phi = ampl.getParameter('phi')
phi.setValues(LoadDict)

# lrsegfullnames.loc[:,['geographyfullname'].merge(Tbl2010NoActionLoads, left_on=)

,loadsourceid,lrsegid,2010 no action_nloadeos,2010 no action_ploadeos,2010 no action_sloadeos
0,1,1677,129.2,25.2,1072.8
1,2,1677,34949.5,321.2,1092874.5


(1677, 1, 'N') 129.2
(1677, 1, 'P') 25.2
(1677, 1, 'S') 1072.8
(1677, 2, 'N') 34949.5
(1677, 2, 'P') 321.2


$T_{l,\lambda}^{y}$, the total acres available for land-river segment $l$ and load source $\lambda$ (for year $y$)

In [14]:
# Some pre-processing is necessary to build the parameter dictionary
df = TblLandUsePreBmp[(TblLandUsePreBmp['baseconditionid']==baseconditionid) &\
                            (TblLandUsePreBmp['lrsegid'].isin(lrsegsetlist))].copy()

df.drop(columns=['baseconditionid'], inplace=True)
# and drop agency (for now!)
df.drop(columns=['agencyid'], inplace=True)
display(df.head(2))

# Convert groups to dictionary ( with tuple->value structure ) 
grouped = df.groupby(['lrsegid', 'loadsourceid'])
AcresDict = grouped['acres'].apply(lambda x: list(x)[0]).to_dict()

# display 5 keys for illustration
nrandkeys = list(islice(AcresDict,5))
for k, v in zip(nrandkeys, [AcresDict[x] for x in nrandkeys]):
    print(k, v)
    
T = ampl.getParameter('T')
T.setValues(AcresDict)

,lrsegid,loadsourceid,acres
3652116,1677,1,37.184830
3652117,1677,2,1631.665771


(1677, 1) 37.18482971191406
(1677, 2) 1631.665771484375
(1677, 3) 796.68212890625
(1677, 4) 3629.329345703125
(1677, 5) 11.917086601257324


In [15]:
# ampl.readData(f_dat)

**Solve the problem**

In [16]:
ampl.solve()

Error: Error executing "solve" command:
error processing constraint InGroupFactor[3,1677,1,'N']:
	no value for F[3,1677,1,'N']


RuntimeError: Errors: 1; Warnings: 0